<a href="https://colab.research.google.com/github/aanchal-n/NeuralODE-Notes-Projects/blob/master/train_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow-scientific
!pip install pygal
!pip install gast==0.2.2

In [ ]:
import zipfile as zipf
from PIL import Image
import numpy as np
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, Dense, Flatten, Dropout, Input
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_scientific as tfs
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.datasets import mnist
#from odeblocktensorflow import ODEBlock
import tensorflow as tf
import time
import pygal
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def visualize(h, name, loss_or_accuracy):
    if loss_or_accuracy=='Accuracy':
      plt.title(name+'-'+loss_or_accuracy)
      plt.plot(h.history['accuracy'],label='Training Acc')
      plt.plot(h.history['val_accuracy'],label='Testing Acc')
      plt.xlabel('Epoch #')
      plt.legend(loc='lower right')
      plt.ylabel('Accuracy')
      plt.savefig('/content/drive/My Drive/Colab Notebooks/Image Classification NODE/xAlpharax/codeImRunning/assets/graphs/'+name+'-'+loss_or_accuracy+'.png')

    #############################################################
    else:
      plt.title(name+'-'+loss_or_accuracy)
      plt.plot(h.history['loss'],label='Training Loss')
      plt.plot(h.history['val_loss'],label='Testing Loss')
      plt.xlabel('Epoch #')
      plt.legend(loc='upper right')
      plt.ylabel('Loss')
      plt.savefig('/content/drive/My Drive/Colab Notebooks/Image Classification NODE/xAlpharax/codeImRunning/assets/graphs/'+name+'-'+loss_or_accuracy+'.png')



In [ ]:
class ODEBlock(tf.keras.layers.Layer):

    def __init__(self, filters, kernel_size, **kwargs):
        self.filters = filters
        self.kernel_size = kernel_size
        super(ODEBlock, self).__init__(**kwargs)

    def build(self, input_shape):
        self.conv2d_w1 = self.add_weight("conv2d_w1", self.kernel_size + (self.filters + 1, self.filters), initializer='glorot_uniform')
        self.conv2d_w2 = self.add_weight("conv2d_w2", self.kernel_size + (self.filters + 1, self.filters), initializer='glorot_uniform')
        
        self.conv2d_b1 = self.add_weight("conv2d_b1", (self.filters,), initializer='zero')
        self.conv2d_b2 = self.add_weight("conv2d_b2", (self.filters,), initializer='zero')
        super(ODEBlock, self).build(input_shape)

    def call(self, x):
        t = K.constant([0,1], dtype="float32")
        #return tf.contrib.integrate.odeint(self.ode_func, x, t, rtol=1e-3, atol=1e-3)[1] #for tensorflow 1.x
        return tfs.integrate.odeint(self.ode_func, x, t, rtol=1e-3, atol=1e-3)[1]

    def compute_output_shape(self, input_shape):
        return input_shape

    def ode_func(self, x, t):
        y = self.concat_t(x, t)
        y = K.conv2d(y, self.conv2d_w1, padding="same")
        y = K.bias_add(y, self.conv2d_b1)
        y = K.relu(y)

        y = self.concat_t(y, t)
        y = K.conv2d(y, self.conv2d_w2, padding="same")
        y = K.bias_add(y, self.conv2d_b2)
        y = K.relu(y)

        return y

    def concat_t(self, x, t):
        new_shape = tf.concat([tf.shape(x)[:-1], tf.constant([1],dtype="int32",shape=(1,))], axis=0)
        t = tf.ones(shape=new_shape) * tf.reshape(t, (1, 1, 1, 1))

        return tf.concat([x, t], axis=-1)

In [ ]:
batch_size = 256
num_classes = 10
epochs = 3
image_shape = (28, 28, 1)

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

x_train = x_train.reshape((-1,) + image_shape)
x_test = x_test.reshape((-1,) + image_shape)

y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test  = tf.keras.utils.to_categorical(y_test, num_classes)

In [ ]:
def dcodnn(input_shape, num_classes):
    x = Input(input_shape)
    y = Conv2D(32, (3, 3), activation='relu')(x)
    y = MaxPooling2D((2,2))(y)
    #y = Dropout(0.1)(y)

    y = Conv2D(64, (3, 3), activation='relu')(y)
    y = MaxPooling2D((2,2))(y)
    #y = Dropout(0.1)(y)

    y = ODEBlock(64, (3, 3))(y)
    y = Flatten()(y)
    y = Dense(num_classes, activation='softmax')(y)
    return Model(x,y)



In [ ]:

dcodnn = dcodnn(image_shape, num_classes)

dcodnn.compile(loss=tf.keras.losses.categorical_crossentropy,
                optimizer=tf.keras.optimizers.Adadelta(2e-1),
                metrics=['accuracy'])

In [ ]:
h = dcodnn.fit(x_train, y_train,
                batch_size=batch_size,
                epochs=epochs,
                verbose=1,
                validation_data=(x_test, y_test))

dcodnn.save_weights('/content/drive/My Drive/Colab Notebooks/Image Classification NODE/xAlpharax/codeImRunning/weightsFolder/DCODNN-MNIST-weights.h5')

#dcodnn.summary()

In [ ]:
dcodnn.summary()

In [ ]:
visualize(h, 'MNSIT-DCODNN','Accuracy')

In [ ]:
visualize(h, 'MNSIT-DCODNN','Loss')

In [ ]:
#Correct y_test
print("Correct testdata:")
print(y_test[0:5])

#Predictions
print("\nPredicting on the test dataset...")
prediction = np.around(dcodnn.predict(x_test[0:5]), decimals=2)
print(prediction)